In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import copy

In [4]:
#This was the process used to load and clean the corpus, and produce the corrupted corpus.
#You don't need to do anything here.
"""
corpus = []
f = open('alice_in_wonderland.txt','r')
while(1):
    line =  f.readline()
    if len(line) == 0: break
    corpus.extend(line.split())
        
f.close()


def clean_word(word):
    word = word.lower().strip()
    for punctuation in ['*','"',"'",'.',',','-','?','!',';',':','—','(',')','[',']']:
        
        word = ''.join(word.split(punctuation))
    
    return word

corpus = [clean_word(word) for word in corpus]
corpus = [word for word in corpus if len(word) > 0]

corrupted_corpus = copy.deepcopy(corpus)

p = .25
alphabet = 'abcdefghijklmnopqrstuvwxyz'
for k in xrange(len(corrupted_corpus)):
    word = corrupted_corpus[k]
    if len(word) < 2: continue
    if np.random.rand() < p:
        if np.random.randn() < 0:
            swap = np.random.choice(range(len(word)), size=2, replace = False)
            swap = np.sort(swap)
            word = ''.join([word[:swap[0]], word[swap[1]], word[swap[0]+1:swap[1]], word[swap[0]], word[swap[1]+1:]])
        else:
            
            replace = np.random.choice(range(len(word)), size=1, replace = False)[0]
            replace_with = alphabet[np.random.choice(range(len(alphabet)),size=1)[0]]
            word = ''.join([word[:replace], replace_with, word[replace+1:]])
        
        corrupted_corpus[k] = word

pickle.dump({'corpus':corpus,'corrupted_corpus':corrupted_corpus},open('alice_spelling.pkl','wb'))
"""

'\ncorpus = []\nf = open(\'alice_in_wonderland.txt\',\'r\')\nwhile(1):\n    line =  f.readline()\n    if len(line) == 0: break\n    corpus.extend(line.split())\n        \nf.close()\n\n\ndef clean_word(word):\n    word = word.lower().strip()\n    for punctuation in [\'*\',\'"\',"\'",\'.\',\',\',\'-\',\'?\',\'!\',\';\',\':\',\'\xe2\x80\x94\',\'(\',\')\',\'[\',\']\']:\n        \n        word = \'\'.join(word.split(punctuation))\n    \n    return word\n\ncorpus = [clean_word(word) for word in corpus]\ncorpus = [word for word in corpus if len(word) > 0]\n\ncorrupted_corpus = copy.deepcopy(corpus)\n\np = .25\nalphabet = \'abcdefghijklmnopqrstuvwxyz\'\nfor k in xrange(len(corrupted_corpus)):\n    word = corrupted_corpus[k]\n    if len(word) < 2: continue\n    if np.random.rand() < p:\n        if np.random.randn() < 0:\n            swap = np.random.choice(range(len(word)), size=2, replace = False)\n            swap = np.sort(swap)\n            word = \'\'.join([word[:swap[0]], word[swap[1]], w

In [2]:
#Take a look at how the data looks, and let's make some helper functions.
data = pickle.load(open('alice_spelling.pkl','rb'))
vocab = np.unique(data['corpus'])
V = len(vocab)


## CORRECT VS INCORRECT CORPUS
##For now, we will hold onto both the correct and incorrect corpuses. Later, you will only process the incorrect corpus, and the correct corpus is only used as a reference to check for recovery accuracy.
def recovery_rate(new_corpus, correct_corpus):
    wrong = 0
    for k in xrange(len(new_corpus)):
        if new_corpus[k] != correct_corpus[k]:
            wrong += 1
    return 1.- wrong/(len(new_corpus)+0.)
print ('current recovery rate', recovery_rate(data['corpus'],data['corrupted_corpus'] ))

## Probability of a word mispelling
## We will use the following function to predict whether a misspelled word was actually another word. To avoid numerical issues, we make sure that the probablity is always something nonzero.
def prob_correct(word1,word2):
    SMALLNUM = 0.000001
    if len(word1) != len(word2): return SMALLNUM
    num_wrong = np.sum(np.array([word1[k] == word2[k] for k in xrange(len(word1))]))
    return np.maximum(num_wrong / (len(word1) + 0.),SMALLNUM)

print ('prob not misspelling alice vs alace', prob_correct('alice','alace'))
print ('prob not misspelling alice vs earth', prob_correct('alice','earth'))
print ('prob not misspelling machinelearning vs machinedreaming', prob_correct('machinelearning','machinedreaming'))
print ('prob not misspelling machinelearning vs artificalintell', prob_correct('machinelearning','artificalintell'))

##HASHING
#all of our objects should be vectors of length V or matrices which are V x V. 
#the kth word in the vocab list is represented by the kth element of the vector, and the relationship between the i,jth words is represented in the i,jth element in the matrix.
# to easily go between the word indices and words themselves, we need to make a hash table. 
vocab_hash = {}
for k in xrange(len(vocab)):
    vocab_hash[vocab[k]] = k
    
#now, to access the $k$th word, we do vocab[k]. To access the index of a word, we call vocab_hash[word].




('current recovery rate', 0.7716434266712013)
('prob not misspelling alice vs alace', 0.80000000000000004)
('prob not misspelling alice vs earth', 9.9999999999999995e-07)
('prob not misspelling machinelearning vs machinedreaming', 0.66666666666666663)
('prob not misspelling machinelearning vs artificalintell', 9.9999999999999995e-07)


In [3]:
## FILL ME IN ##

#WORD FREQUENCY
#create an array of length V where V[k] returns the normalized frequency of word k in the entire data corpus. Do so by filling in this function.
def get_word_prob(corpus):
    word_prob = np.ones(V, dtype=np.float64)/(V+0.)
    corpus_hash = list(map(lambda x: vocab_hash[x], corpus))
    counts = np.unique(corpus_hash, return_counts=True)
    counts = list(zip(counts[0],counts[1]))
    for i, j in counts:
        word_prob[i] = j/(len(corpus)+0.)
    return word_prob

word_prob =  get_word_prob(data['corpus'])

#report the answer of the following:
print 'prob. of "alice"', word_prob[vocab_hash['alice']]
print 'prob. of "queen"', word_prob[vocab_hash['queen']]
print 'prob. of "chapter"', word_prob[vocab_hash['chapter']]




# Pr(word | prev word) 
#Using the uncorrupted corpus, accumulate the conditional transition probabilities. Do so via this formula:
# pr(word | prev) = max(# times 'prev' preceded 'word' , 1) / # times prev appears
# where again, we ensure that this number is never 0 with some small smoothing.
def get_transition_matrix(corpus):

    temp_transition_matrix = np.zeros((len(vocab),len(vocab)), dtype=np.float64)
    transition_matrix = np.zeros((len(vocab),len(vocab)), dtype=np.float64)
    corpus_hash = np.array(list(map(lambda x: vocab_hash[x],corpus))).reshape(-1, 1)
    #[I, Am, Adithya, Ganesan] -> [1,2,3,4]
    prev_word_hash_matrix = np.concatenate([corpus_hash[:-1], corpus_hash[1:]], axis=1)
    #[1,2,3,4] -> [[1,2], [2,3], [3,4]]
    
    for prev, word in prev_word_hash_matrix:
        temp_transition_matrix[word, prev] = temp_transition_matrix[word, prev]+1 
    #temp_transition_matrix[transition_matrix==0] = 1

    for i in range(V):
        #transition_matrix[:, i] /= (corpus.count(i))
        transition_matrix[:, i] = temp_transition_matrix[:, i]/(word_prob[i]*len(corpus))

    return transition_matrix

transition_matrix = get_transition_matrix(data['corpus'])
print 'prob. of "the alice"', transition_matrix[vocab_hash['alice'],vocab_hash['the']]
print 'prob. of "the queen"', transition_matrix[vocab_hash['queen'],vocab_hash['the']]
print 'prob. of "the chapter"', transition_matrix[vocab_hash['chapter'],vocab_hash['the']]
print 'prob. of "the hatter"', transition_matrix[vocab_hash['hatter'],vocab_hash['the']]

prob. of "alice" 0.0145486150474
prob. of "queen" 0.00256962551487
prob. of "chapter" 0.000906926652307
prob. of "the alice" 0.0
prob. of "the queen" 0.0396825396825
prob. of "the chapter" 0.0
prob. of "the hatter" 0.0311355311355


In [4]:
#The prior probabilities are just the word frequencies
prior = word_prob  + 0.

#write a function that returns the emission probability of a potentially misspelled word, by comparing its probabilities against every word in the correct vocabulary
def get_emission(mword):
    emission = np.empty((V,))
    for word in vocab:
        emission[vocab_hash[word]] = prob_correct(word, mword)
    return emission

#find the 10 closest words to 'abice' and report them
idx = np.argsort(get_emission('abice'))[::-1]
print [vocab[j] for j in idx[:10]]


['abide', 'alice', 'above', 'voice', 'alive', 'twice', 'thick', 'dance', 'stick', 'prize']


In [5]:
#now we reduce our attention to a small segment of the corrupted corpus
corrupt_corpus =   data['corrupted_corpus'][:1000]
correct_corpus =   data['corpus'][:1000]

In [6]:
print (list(zip(corrupt_corpus, correct_corpus))[:20])

[('alices', 'alices'), ('adventures', 'adventures'), ('in', 'in'), ('wonderland', 'wonderland'), ('yb', 'by'), ('lewia', 'lewis'), ('carroll', 'carroll'), ('the', 'the'), ('millennium', 'millennium'), ('fulcrkm', 'fulcrum'), ('edition', 'edition'), ('30', '30'), ('contents', 'contents'), ('chapter', 'chapter'), ('i', 'i'), ('down', 'down'), ('tqe', 'the'), ('raibbthole', 'rabbithole'), ('chapter', 'chapter'), ('ii', 'ii')]


In [7]:
forward_probs = np.ones((len(corrupt_corpus), V))/(V+0.)*0
for t, word in enumerate(corrupt_corpus):
    if t != 0:
        emission = get_emission(word)
        for k in range(V):
            forward_probs[t, :] += (emission*transition_matrix[:,k]*forward_probs[t-1, k])
    else:
        forward_probs[t, :] = prior#(get_emission(word)*prior)
    forward_probs[t, :] = forward_probs[t, :]/ np.sum(forward_probs[t, :])

In [8]:
proposed_corpus = []
for j, word in enumerate(corrupt_corpus):
    corrected_word = vocab[np.argsort(forward_probs[j])[-1]]
    proposed_corpus.append(corrected_word)
print recovery_rate(corrupt_corpus, correct_corpus), recovery_rate(proposed_corpus, correct_corpus)
#for k in xrange(100):
    #print proposed_corpus[k], corrupt_corpus[k]

0.759 0.797


In [9]:
backward_probs = np.ones((len(corrupt_corpus), V))/(V+0.)*0
T = len(corrupt_corpus)
for t in reversed(range(T)):
    if t == T-1:
        #backward_probs[t, :] = (get_emission(corrupt_corpus[t]))
        #backward_probs[t, :] =  np.sum(get_emission(corrupt_corpus[t])*transition_matrix, axis=1)
        backward_probs[t, :] = np.ones(backward_probs[t, :].shape)
    else:
        #for k in range(V):
            #backward_probs[t, :] += (transition_matrix[k, :]*get_emission(corrupt_corpus[t+1])*backward_probs[t+1, j])
        temp = (backward_probs[t+1]*get_emission(corrupt_corpus[t+1])).reshape(1, -1)
        backward_probs[t, :] = np.dot(temp, transition_matrix).reshape(-1, )
        backward_probs[t, :] = backward_probs[t, :]/ np.sum(backward_probs[t, :])

In [10]:
proposed_corpus = []
for j, word in enumerate(corrupt_corpus):
    corrected_word = vocab[np.argsort(backward_probs[j])[-1]]
    proposed_corpus.append(corrected_word)
print recovery_rate(corrupt_corpus, correct_corpus), recovery_rate(proposed_corpus, correct_corpus)
for k in xrange(-100, 0):
    print proposed_corpus[k], corrupt_corpus[k]

0.759 0.078
reduced in
important the
weeks air
treaclewelleh im
suet afraid
advise but
you you
dreaming might
thunderstorm zatch
cheshire a
zigzag bat
romeno and
tricks thats
shaped very
vii yike
english a
washingextra uomse
muchnessyou yot
cheshire know
airs but
hate do
shell cats
fifth eat
egg bats
turkey i
raving wodner
forehead and
fitted here
theyve alice
rising began
shell to
fishand get
languid rather
zigzag sleepy
hurriedly and
eel went
paw on
fancywhos saying
raised zo
snatch herswlf
thunderstorm in
a a
helpless dregmy
parts sort
common of
airs way
hated do
shell cast
hate eat
likes bats
hated od
shell cats
elses eat
zigzag bats
growing and
airs uometimes
hate do
shell tabs
hate eat
cares cats
couple ror
dripping yuo
curtseying see
whom as
they she
couldnt couldnt
visit answer
difficult either
uncorked question
yours it
signifies didnt
ma hucm
writingdesks matter
common which
busy way
setting she
beor put
afore it
archbishop she
thingseverything felt
xi that
fortunately ehs
gr

In [11]:
probs = forward_probs*backward_probs
proposed_corpus = []
for j, word in enumerate(corrupt_corpus):
    corrected_word = vocab[np.argsort(probs[j])[-1]]
    proposed_corpus.append(corrected_word)
print recovery_rate(corrupt_corpus, correct_corpus), recovery_rate(proposed_corpus, correct_corpus)
for k in xrange(100):
    print proposed_corpus[k], corrupt_corpus[k]

0.759 0.906
a alices
adventures adventures
in in
wonderland wonderland
by yb
lewis lewia
carroll carroll
the the
millennium millennium
fulcrum fulcrkm
edition edition
30 30
contents contents
chapter chapter
i i
dont down
the tqe
rabbithole raibbthole
chapter chapter
ii ii
the the
pool pooo
of of
tears aetrs
chapter chapter
iii dii
a a
caucusrace caucusrace
and and
a a
long long
tale tael
chapter yhapter
iv iv
the the
rabbit raibbt
sends sends
in ni
a a
little littme
bill bill
chapter chapter
v v
advice advice
from from
a a
caterpillar raterpillac
chapter chapter
vi vi
pig piz
and xnd
pepper pepper
chapter chapter
vii vii
a a
mad amd
teaparty teaparty
chapter chapter
viii viii
the the
queens zueens
croquetground croquetground
chapter chapter
iv ic
the the
mock mock
turtles turtles
story story
chapter chapter
x x
the the
lobster lsboter
quadrille quadrille
chapter chartep
xi xi
who who
stole stole
the eht
tarts tarts
chapter chapter
xii xii
alices alices
evidence nvideece
chapter chapter

In [35]:
import pandas as pd

In [36]:
corpora = np.concatenate([np.array(proposed_corpus).reshape(-1,1), np.array(corrupt_corpus).reshape(-1,1), np.array(correct_corpus).reshape(-1,1)], axis=1)

In [41]:
a = corpora[(corpora[:, 1] != corpora[:, 2])&(corpora[:, 2] != corpora[:, 0])]
print (a.shape)

(36, 3)


In [42]:
print(pd.DataFrame(a, columns=["Proposed", "Corrupt", "Correct"]).to_latex(index=False))

\begin{tabular}{lll}
\toprule
Proposed &  Corrupt &  Correct \\
\midrule
      iv &       ic &       ix \\
    dont &     donw &     down \\
      to &       fo &       of \\
      to &       ro &       or \\
   alice &    twcce &    twice \\
      or &       on &       no \\
      to &       fo &       of \\
 getting &  nlthing &  nothing \\
     was &      bus &      but \\
      to &       ta &       at \\
      to &       ti &       it \\
      to &       ti &       it \\
 herself &  flasred &  flashed \\
  before &   caross &   across \\
    make &     taze &     take \\
      as &       af &       of \\
      to &       ti &       it \\
    with &     lika &     like \\
    that &     thne &     then \\
      to &       ta &       at \\
      so &       sa &       as \\
    with &     nito &     into \\
      to &       fo &       of \\
      or &       ot &       to \\
      so &       fo &       of \\
      to &       ta &       at \\
   after &    nrvee &    never \\
    some 

In [44]:
b = corpora[(corpora[:, 1] != corpora[:, 2])&(corpora[:, 2] == corpora[:, 0])]
print b.shape


(205, 3)


In [45]:
r = np.random.randint(0, b.shape[0], size=(36,))
print(pd.DataFrame(b[r], columns=["Proposed", "Corrupt", "Correct"]).to_latex(index=False))

\begin{tabular}{lll}
\toprule
    Proposed &      Corrupt &      Correct \\
\midrule
        this &         bhis &         this \\
          on &           cn &           on \\
          so &           ss &           so \\
       heads &        hdaes &        heads \\
        this &         bhis &         this \\
        very &         vhry &         very \\
        once &         onec &         once \\
       didnt &        dindt &        didnt \\
      wonder &       wodner &       wonder \\
         the &          thd &          the \\
          no &           qo &           no \\
          it &           iv &           it \\
         hot &          hto &          hot \\
         the &          tge &          the \\
         say &          soy &          say \\
         had &          hat &          had \\
         ask &          aik &          ask \\
      queens &       zueens &       queens \\
     dinahll &      diwahll &      dinahll \\
          up &           cp &           u

# encode the HMM spelling corrector. To debug, you can see the first hundred words of both the corrupted and proposed corpus, to see if spelling words got corrupted.

# report the recovery rate of the proposed (corrected) corpus.

#forward step
forward_probs = np.ones((len(corrupt_corpus), V))/(V+0.)*0
for t, word in enumerate(corrupt_corpus):
    if t != 0:
        emission = get_emission(word)
        for k in range(V):
            forward_probs[t, :] += (emission*transition_matrix[:,k]*forward_probs[t-1, k])
        forward_probs[t, :] = forward_probs[t, :]/ np.sum(forward_probs[t, :])
    else:
        forward_probs[t, :] = (get_emission(word)*prior)
        forward_probs[t, :] = forward_probs[t, :]/ np.sum(forward_probs[t, :])
    
# backward step
backward_probs = np.ones((len(corrupt_corpus), V))/(V+0.)*0


# compute corrected corpus
proposed_corpus = copy.deepcopy(corrupt_corpus)
for k in xrange(100):
    print proposed_corpus[k], corrupt_corpus[k]

print recovery_rate(corrupt_corpus, correct_corpus), recovery_rate(proposed_corpus, correct_corpus) 

    

    